# Chapter 4: MCP Integration with Strands Agents

## Introduction to Model Context Protocol (MCP)

One of the most powerful features of Strands Agents is its ability to integrate with the Model Context Protocol (MCP). This integration allows your agents to access a wide range of external tools and resources, significantly expanding their capabilities beyond what's available in the standard toolset.

<img src="mcp.png" width="750" alt="mcp">

Throughout this chapter, we'll use the Nova Lite model (`us.amazon.nova-lite-v1:0`) as specified for our course.

## What is the Model Context Protocol?

The Model Context Protocol (MCP) is a standardized way for AI models to interact with external tools and resources. It defines a common interface for exposing functionality to AI systems, allowing for seamless integration between different platforms and services.

Key features of MCP include:

1. **Standardized Communication:** MCP provides a common language for AI models and tools to communicate
2. **Tool Discovery:** MCP servers can expose multiple tools, and clients can discover available tools
3. **Resource Access:** Beyond tools (functions), MCP also supports accessing resources like files, databases, or APIs
4. **Transport Independence:** MCP works over various [transport mechanisms](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports) (Streamable HTTP, stdio, etc.)
5. **Cross-Platform Compatibility:** Tools from one platform can be used by agents on different platforms

## Setup and Prerequisites

Let's start by installing the necessary packages for MCP integration:

In [ ]:
# Install strands-agents and strands-agents-tools if you haven't already
!pip install -U strands-agents strands-agents-tools

# Install mcp for MCP client/server implementation
!pip install -U mcp

## Connecting to MCP Servers

Strands Agents can connect to MCP servers using the `MCPClient` class from the `strands.tools.mcp` module. The most common approach is to use the Standard I/O (stdio) transport mechanism:

In [ ]:
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.tools.mcp import MCPClient

# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"])
))

# The with statement ensures the client is properly initialized and later cleaned up
with stdio_mcp_client:
    # Get the tools from the MCP server
    mcp_tools = stdio_mcp_client.list_tools_sync()
    
    # Print the available tools for inspection
    for tool in mcp_tools:
        print(f"Tool: {tool.tool_name}")
        print(f"Description: {tool.tool_spec}")
        print("\n")

In [ ]:
# Another option using MCP - Start MCP server without 'with'
stdio_mcp_client.start()

# Get the tools from the MCP server
mcp_tools = stdio_mcp_client.list_tools_sync()

# Print the available tools for inspection
for tool in mcp_tools:
    print(f"Tool: {tool.tool_name}")
    print(f"Description: {tool.tool_spec}")
    print("\n")

# Stop MCP server
stdio_mcp_client.stop()

## Using MCP Tools in Agents

Once we have connected to an MCP server and obtained its tools, we can create an agent that uses these tools:

In [ ]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    mcp_tools = stdio_mcp_client.list_tools_sync()
    
    # Create an agent with these tools
    aws_docs_agent = Agent(
        model="us.amazon.nova-pro-v1:0",
        tools=mcp_tools,
        system_prompt="You are a helpful assistant that specializes in AWS documentation."
    )
    
    # Ask the agent a question that will make use of the MCP tools
    response = aws_docs_agent("""
    Can you explain how to set up an Amazon S3 bucket with versioning enabled? 
    Please provide detailed steps based on the official AWS documentation.
    """)

## Combining MCP Tools with Built-in and Custom Tools

One of the great advantages of Strands Agents is the ability to combine different types of tools:

In [ ]:
from strands import tool
from strands_tools import calculator, python_repl

@tool
def note_taker(note: str) -> str:
    """
    Save a note to the notes collection.
    
    Args:
        note (str): The content of the note to save
        
    Returns:
        str: Confirmation message with timestamp
    """
    import datetime
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"Note saved at {timestamp}: {note}"

# Create an agent with a mix of tool types
with stdio_mcp_client:
    # Get the MCP tools
    mcp_tools = stdio_mcp_client.list_tools_sync()
    
    # Create an agent with mixed tools
    hybrid_agent = Agent(
        model="us.amazon.nova-pro-v1:0",
        tools=[
            *mcp_tools,         # MCP tools
            calculator,         # Built-in tool
            python_repl,        # Built-in tool
            note_taker          # Custom tool
        ],
        system_prompt="You are a versatile assistant that can search documentation, perform calculations, and take notes."
    )

## Creating Your Own MCP Server

You can also create your own MCP server to expose custom functionality:

In [ ]:
%%writefile weather_mcp.py
from mcp.server.fastmcp import FastMCP

# Initialize FastMCP server
mcp = FastMCP("demo-weather")

@mcp.tool()
async def get_weather(city: str) -> str:
    """Get demo weather for a city (simplified example)"""
    demo_data = {
        "Beijing": "Sunny, 25°C",
        "Shanghai": "Cloudy, 23°C",
        "New York": "Rainy, 18°C"
    }
    return demo_data.get(city, f"Weather data not available for {city}")

if __name__ == "__main__":
    # Run server in stdio mode (simplest for demo)
    mcp.run(transport='stdio')

In [ ]:
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.tools.mcp import MCPClient

# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(command="python", args=["weather_mcp.py"])
))

# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    mcp_tools = stdio_mcp_client.list_tools_sync()
    
    # Create an agent with these tools
    weather_mcp_agent = Agent(
        model="us.amazon.nova-pro-v1:0",
        tools=mcp_tools
    )

    weather_mcp_agent("How's the weather in Beijing?")

## Best Practices for MCP Integration

When working with MCP in Strands Agents, consider these best practices:

1. **Clear Tool Descriptions**: Provide detailed descriptions so the agent knows when and how to use the tool
2. **Graceful Error Handling**: Return informative error messages to help the agent recover
3. **Well-defined Schemas**: Use schemas to specify required inputs and expected outputs
4. **Focused Tools**: Create specialized tools rather than multi-purpose "Swiss Army knife" tools
5. **Complementary Design**: Design MCP tools to work well with other tool types

## Summary

In this chapter, we've explored how to integrate Strands Agents with the Model Context Protocol (MCP). Key takeaways include:

1. MCP provides a standardized way for AI models to interact with external tools and resources
2. Strands Agents can connect to MCP servers using the `MCPClient` class
3. MCP tools can be used alongside built-in and custom tools in Strands Agents
4. You can create your own MCP servers to expose custom functionality
5. Best practices include clear tool descriptions, robust error handling, and focused tool design

In the next chapter, we'll explore deploying Strands Agents to production environments, with a focus on AWS Lambda deployments.

## Exercises

1. Create an MCP server that exposes a tool for translating text between languages
2. Create an MCP client that connects to an existing MCP server and uses its tools
3. Create an agent that uses a mix of MCP tools, built-in tools, and custom tools
4. Explore the MCP protocol specification and identify other transport mechanisms besides stdio
5. Design an MCP server that integrates with a database or other backend service